In [1]:
import os
import tensorflow as tf
# sess_cpu = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(device_count={'GPU': 0}))
from tensorflow import keras
import tensorflow.keras.backend as K
from skimage.io import imread
from skimage.transform import resize
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
tf.keras.backend.clear_session()  # For easy reset of notebook state.

Using TensorFlow backend.


In [2]:
IMG_SIZE = 165
CROP_SIZE = 150
DS_SIZE = 6000
EPS = 1e-8
BATCH_SIZE = 30
MAX_SHIFT = IMG_SIZE - CROP_SIZE
ROTATE_MAX = 15
half = MAX_SHIFT // 2
mirror_map = [3, 2, 1, 0, 9, 8, 7, 6, 5, 4, 10, 13, 12, 11]

In [3]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

In [4]:
get_available_gpus()

['/device:GPU:0']

In [5]:
from tensorflow.keras import Sequential
from tensorflow.keras import layers
# from tensorflow.keras.layers import Dense, Activation, ReLU, Conv2D, MaxPool2D, BatchNormalization, Dense

In [6]:
def load_and_resize(filename, Y, num):
    image = imread(filename)
    hei = image.shape[0]
    wid = image.shape[1]
    Y[i,::2] = (Y[i,::2] * IMG_SIZE) / wid
    Y[i,1::2] = (Y[i,1::2] * IMG_SIZE) / hei
    image = resize(image, (IMG_SIZE, IMG_SIZE))
    if len(image.shape) == 2:
        image = np.stack((image, image, image))
        image = np.transpose(image, axes=(1,2, 0))
#     print(image.shape)
    for channel in range(3):
        image[:,:,channel] = (image[:,:,channel] - np.mean(image[:,:,channel])) / (np.std(image[:,:,channel]) + EPS)
#     print(image.shape)
    return image

In [7]:
def read_gt():
#     csv_file = csv.DictReader(open('public_tests/00_test_img_input/train/gt.csv'))
#     lines = [line for line in csv_file]
    lines = pd.read_csv('public_tests/00_test_img_input/train/gt.csv')
    lines.drop('filename', axis=1, inplace=True)
    return lines

In [8]:
# lines = read_gt()[:100]
# print(lines)

In [9]:
# for i in range(200, 300):
#     print(i)
#     X[i] = 
# img = load_and_resize('public_tests/00_test_img_input/train/images/{:05}.jpg'.format(20))
# print(img.shape)
# print(img[0, 0, :])

In [10]:
X = np.zeros((DS_SIZE, IMG_SIZE, IMG_SIZE, 3))
Y = read_gt()[:DS_SIZE].values.astype(float)
for i in range(DS_SIZE):
#     print(Y[i])
    X[i] = load_and_resize('public_tests/00_test_img_input/train/images/{:05}.jpg'.format(i), Y, i)
#     print(Y[i])
    if i % 500 == 0:
        print('{} loaded'.format(i))
#     print('public_tests/00_test_img_input/train/images/{:05}.jpg'.format(i))
    

0 loaded
500 loaded
1000 loaded
1500 loaded
2000 loaded
2500 loaded
3000 loaded
3500 loaded
4000 loaded
4500 loaded
5000 loaded
5500 loaded


In [12]:
print(np.count_nonzero(Y < 20) / 168000)

0.006208333333333333


In [13]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

In [20]:
model = keras.Sequential()
model.add(layers.Conv2D(80, kernel_size=(3,3), input_shape=(CROP_SIZE, CROP_SIZE, 3)))
model.add(layers.BatchNormalization())
model.add(layers.ReLU())
model.add(layers.MaxPool2D())

model.add(layers.Conv2D(160, kernel_size=(3,3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.ReLU())
model.add(layers.MaxPool2D())

model.add(layers.Conv2D(320, kernel_size=(3,3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.ReLU())
model.add(layers.MaxPool2D())

model.add(layers.Flatten())
# model.add(layers.Dropout(0.2))
model.add(layers.Dense(80, activation='relu'))
# model.add(layers.BatchNormalization())
model.add(layers.Dense(28))
# model = keras.models.load_model('dropout_model.h5')

In [21]:
import random
from skimage.transform import SimilarityTransform
from skimage.transform import warp

def rotate_transform_matrix(transform):
    """Rotate matrix so it can be applied to row:col coordinates."""
    matrix = transform.params[(1, 0, 2), :][:, (1, 0, 2)]
    return type(transform)(matrix)

class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, images, points, batch_size=BATCH_SIZE,
                 shuffle=True, seed=None):
        'Initialization'
        self.batch_size = batch_size
        self.points = points
        self.images = images
        self.shuffle = shuffle
        self.on_epoch_end()
        if seed:
            random.seed(20)

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.images) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        image_batch = [self.images[k] for k in indexes]
        points_batch = [self.points[k] for k in indexes]
        

        # Generate data
        X, y = self.__data_generation(image_batch, points_batch)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.images))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, image_batch, points_batch):
        'Generates data containing batch_size samples'
        # Initialization
        X = np.empty((self.batch_size, CROP_SIZE, CROP_SIZE, 3))
        y = np.empty((self.batch_size,*points_batch[0].shape))

        # Generate data
        for i in range(self.batch_size):
            # Store sample
            mirror = bool(random.getrandbits(1))
            angle = random.random() * 2 * ROTATE_MAX - ROTATE_MAX
            center = np.array((IMG_SIZE, IMG_SIZE)) / 2. - 0.5
            tform1 = SimilarityTransform(translation=center)
            tform2 = SimilarityTransform(rotation=np.deg2rad(angle))
            tform3 = SimilarityTransform(translation=-center)
            tform = tform3 + tform2 + tform1
            tform.params[2] = (0, 0, 1)
            rotated_img = warp(image_batch[i], rotate_transform_matrix(tform).inverse)
            if mirror:
                rotated_img = np.fliplr(rotated_img)
#             rotated_img = image_batch[i]
            
            dx = half # random.randint(0, MAX_SHIFT)
            dy = half # random.randint(0, MAX_SHIFT)
#             print(f"{dx} {dy}")
            X[i,] = rotated_img[dx:dx+CROP_SIZE,dy:dy+CROP_SIZE,:]
            for p_num in range(14):
                rotated_point = tform([points_batch[i][p_num*2+1], points_batch[i][p_num*2]])
#                 rotated_point = [[points_batch[i][p_num*2+1], points_batch[i][p_num*2]]]
                if mirror:
                    y[i,mirror_map[p_num]*2+1] = (rotated_point[0][0]-dy) / CROP_SIZE - 0.5
                    y[i,mirror_map[p_num]*2] = (CROP_SIZE - (rotated_point[0][1]-dx)) / CROP_SIZE - 0.5
                else:
                    y[i,p_num*2+1] = (rotated_point[0][0]-dy) / CROP_SIZE - 0.5
                    y[i,p_num*2] = (rotated_point[0][1]-dx) / CROP_SIZE - 0.5
                

            # Store class
#             y[i,::2] = points_batch[i][::2] - dx
#             y[i,1::2] = points_batch[i][1::2] - dy
            

        return X, y

In [22]:
# train_datagen = ImageDataGenerator()
# train_datagen.fit(x_train)
# train_generator = train_datagen.flow(x_train, y_train, batch_size=BATCH_SIZE)
# step = x_train.shape[0] // BATCH_SIZE

train_generator = DataGenerator(x_train, y_train, seed=1)

In [23]:
adam = keras.optimizers.Adam(learning_rate=0.005)

In [24]:
model.compile(optimizer='adam',loss='mean_squared_error', metrics=['accuracy'])

In [25]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 148, 148, 80)      2240      
_________________________________________________________________
batch_normalization_3 (Batch (None, 148, 148, 80)      320       
_________________________________________________________________
re_lu_3 (ReLU)               (None, 148, 148, 80)      0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 74, 74, 80)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 72, 72, 160)       115360    
_________________________________________________________________
batch_normalization_4 (Batch (None, 72, 72, 160)       640       
_________________________________________________________________
re_lu_4 (ReLU)               (None, 72, 72, 160)      

In [ ]:
history = model.fit_generator(
    train_generator, 
#     steps_per_epoch=step, 
    epochs=50,
    validation_data=(x_test[:, half:half+CROP_SIZE, half:half+CROP_SIZE, :], (y_test - half) / CROP_SIZE - 0.5))

Epoch 1/50
160/160 [==============================] - 66s 411ms/step - loss: 7.8750 - accuracy: 0.7667 - val_loss: 0.0405 - val_accuracy: 0.8175
Epoch 2/50
160/160 [==============================] - 63s 392ms/step - loss: 0.0362 - accuracy: 0.7754 - val_loss: 0.0321 - val_accuracy: 0.8175
Epoch 3/50
160/160 [==============================] - 63s 392ms/step - loss: 0.0287 - accuracy: 0.7835 - val_loss: 0.0253 - val_accuracy: 0.8175
Epoch 4/50
160/160 [==============================] - 63s 391ms/step - loss: 0.0228 - accuracy: 0.7806 - val_loss: 0.0200 - val_accuracy: 0.8175
Epoch 5/50
160/160 [==============================] - 63s 392ms/step - loss: 0.0181 - accuracy: 0.7804 - val_loss: 0.0158 - val_accuracy: 0.8175
Epoch 6/50
160/160 [==============================] - 63s 392ms/step - loss: 0.0144 - accuracy: 0.7908 - val_loss: 0.0126 - val_accuracy: 0.8175
Epoch 7/50
160/160 [==============================] - 63s 393ms/step - loss: 0.0117 - accuracy: 0.7796 - val_loss: 0.0101 - val_ac

In [65]:
history = model.fit_generator(
    train_generator, 
#     steps_per_epoch=step, 
    epochs=50,
    validation_data=(x_test[:, half:half+CROP_SIZE, half:half+CROP_SIZE, :], (y_test - half) / CROP_SIZE - 0.5))

Epoch 1/50
80/80 [==============================] - 36s 453ms/step - loss: 7.7407e-04 - accuracy: 0.8244 - val_loss: 0.0011 - val_accuracy: 0.8608
Epoch 2/50
80/80 [==============================] - 36s 447ms/step - loss: 7.4610e-04 - accuracy: 0.8210 - val_loss: 0.0011 - val_accuracy: 0.8617
Epoch 3/50
80/80 [==============================] - 36s 446ms/step - loss: 7.6196e-04 - accuracy: 0.8179 - val_loss: 0.0011 - val_accuracy: 0.8583
Epoch 4/50
80/80 [==============================] - 36s 445ms/step - loss: 7.2860e-04 - accuracy: 0.8273 - val_loss: 0.0011 - val_accuracy: 0.8608
Epoch 5/50
80/80 [==============================] - 36s 446ms/step - loss: 7.1773e-04 - accuracy: 0.8277 - val_loss: 0.0011 - val_accuracy: 0.8575
Epoch 6/50
80/80 [==============================] - 36s 446ms/step - loss: 7.0593e-04 - accuracy: 0.8273 - val_loss: 0.0011 - val_accuracy: 0.8625
Epoch 7/50
80/80 [==============================] - 36s 446ms/step - loss: 7.1155e-04 - accuracy: 0.8225 - val_loss: 0

In [33]:
# Save the model
model.save('dropout_mirrored_model.h5')

In [66]:
y_pred = model.predict(x_test[:, half:half+CROP_SIZE, half:half+CROP_SIZE, :])

In [67]:
def compute_metric(pred, test):
    res = 0.0
    hei = pred.shape[0]
    print(pred.shape)
    for i in range(hei):
        diff = (pred[i] - test[i])
#         diff[::2] /= n_cols
#         diff[1::2] /= n_rows
#         diff *= 100
#         print((diff ** 2).mean())
#         diff = diff / CROP_SIZE * 100
        diff *= 100
        res += (diff ** 2).mean()
    return res / hei

print(compute_metric(y_pred, (y_test - half) / CROP_SIZE - 0.5))

(1200, 28)
10.470492994338189


In [115]:
for line in range(50):
#     print(y_pred[line])
#     print(y_test[line])
    print((np.abs(y_pred[line] - y_test[line])).astype(int))

[26 24  5 28  4 25 22 12 19 23 14 23  9 22  5 18 11 18 16 15  5  3 21  3
  9  9  3  7]
[2 7 2 7 1 5 0 3 2 7 1 8 3 7 6 6 5 5 6 4 1 4 4 6 3 6 5 4]
[1 6 2 6 4 3 6 2 3 6 2 6 3 4 0 4 4 3 5 2 2 8 4 4 3 2 7 5]
[ 2  6  3  2  4  5  6 10  5  4  4  4  6  4  7  6  7  6  8  6  3  3  4  0
  4  3  5  2]
[5 5 5 5 4 2 1 3 2 3 3 2 5 3 1 2 4 1 5 0 7 0 3 1 5 0 5 1]
[5 2 6 2 7 4 5 8 5 1 4 2 3 1 6 4 7 4 5 5 6 5 5 0 5 3 8 3]
[2 4 2 6 1 7 5 5 2 5 3 4 4 6 2 4 2 4 3 4 1 2 3 4 2 3 1 3]
[6 4 3 9 4 9 3 6 8 4 6 3 5 5 2 5 2 5 2 6 0 2 2 0 0 0 1 2]
[4 3 2 2 4 2 6 4 5 6 6 6 7 8 3 7 5 6 6 5 5 5 8 6 5 4 4 4]
[4 5 4 9 8 7 6 8 5 4 8 4 5 4 3 4 4 5 4 5 1 7 6 8 3 6 0 7]
[5 7 8 5 5 6 6 9 8 5 7 3 8 5 5 6 5 5 5 5 7 3 5 2 7 2 5 3]
[3 3 6 0 6 0 1 4 7 4 5 5 5 5 2 3 4 5 4 4 1 5 0 3 4 4 4 2]
[4 7 1 5 1 5 1 5 3 5 3 4 6 2 3 4 6 4 6 3 0 4 2 6 3 4 2 3]
[ 7  5  5  0  5  2 13  2 10  1  9  0 11  1 11  3 15  4 17  5  3  4 11  4
  9  1 13  1]
[ 2 20  0 19  1  9 11  3  0 18  0 17  0 12  4  6  6  4  6  1  5  9  0  3
  2  0  2 11]
[4 8 0 7 2 6 3